In [1]:
import json

from gpras.gpr import RasEmulator
from gpras.ras import RasModel

In [2]:
# Constants
ras_path = "data/Muncie/Muncie.prj"
crs = "ESRI:102673"
stac_path = "data/Muncie/Muncie.stac.json"
emulator_meta_path = "data/ras_emulator.json"

In [3]:
# Make initial STAC item. Only need to run once.
stac = RasModel.from_prj(ras_path, crs)
with open(stac_path, mode="w") as f:
    json.dump(stac.to_dict(), f, indent=4)

In [4]:
# Make emulator metadata json.  Only need to run once.
metadata = {
    "ras_model_path": stac_path,
    "hf_mesh_id": "2D Interior Area",
    "lf_mesh_id": "2D Interior Area",
    "lf_runs": ["lf1", "lf2", "lf3"],
    "hf_runs": ["hf1", "hf2", "hf3"],
}
with open(emulator_meta_path, mode="w") as f:
    json.dump(metadata, f, indent=4)

In [5]:
# Start GPR model
with open(emulator_meta_path) as f:
    emulator_meta = json.load(f)
gpr = RasEmulator(**emulator_meta)

In [6]:
gpr.data_x

/workspaces/gpras/gpras/gpr.py:204: UserWarning: `keep_geom_type=True` in overlay resulted in 85 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
  mesh_resampled = gpd.overlay(self.geometry_hf, self.geometry_lf[["cell_id", "geometry"]], how="intersection")


0           1           2           3           4     \
run t                                                                 
0   0    929.312500  929.312500  929.312500  929.312500  930.625000   
    1    929.312500  929.312500  929.312500  929.312500  930.625000   
    2    929.312500  929.312500  929.312500  929.312500  930.625000   
    3    929.312500  929.312500  929.312500  929.312500  930.625000   
    4    929.312500  929.312500  929.312500  929.312500  930.625000   
...             ...         ...         ...         ...         ...   
2   284  945.068237  945.068237  945.068237  945.068237  945.068298   
    285  945.068298  945.068298  945.068298  945.068298  945.068359   
    286  945.068420  945.068420  945.068420  945.068420  945.068420   
    287  945.068481  945.068481  945.068481  945.068481  945.068481   
    288  945.068542  945.068542  945.068542  945.068542  945.068542   

               5           6           7           8           9     ...  \
run t                                                                ...   
0   0    929.312500  929.312500  929.312500  929.312500  929.312500  ...   
    1    929.312500  929.312500  929.312500  929.312500  929.312500  ...   
    2    929.312500  929.312500  929.312500  929.312500  929.312500  ...   
    3    929.312500  929.312500  929.312500  929.312500  929.312500  ...   
    4    929.312500  929.312500  929.312500  929.312500  929.312500  ...   
...             ...         ...         ...         ...         ...  ...   
2   284  945.068237  945.068237  945.068237  945.068237  945.068237  ...   
    285  945.068298  945.068298  945.068298  945.068298  945.068298  ...   
    286  945.068359  945.068359  945.068359  945.068359  945.068359  ...   
    287  945.068420  945.068420  945.068420  945.068420  945.068420  ...   
    288  945.068481  945.068542  945.068542  945.068542  945.068542  ...   

               5381        5382        5383        5384        5385  \
run t                                                                 
0   0    941.437500  938.783691  941.437500  940.156982  942.093750   
    1    941.437500  938.783691  941.437500  940.156982  942.093750   
    2    941.437500  938.783691  941.437500  940.156982  942.093750   
    3    941.437500  938.783691  941.437500  940.156982  942.093750   
    4    941.437500  938.783691  941.437500  940.156982  942.093750   
...             ...         ...         ...         ...         ...   
2   284  946.292725  946.278687  946.292725  946.248962  946.277710   
    285  946.292969  946.278931  946.292969  946.249207  946.277954   
    286  946.293152  946.279175  946.293152  946.249390  946.278198   
    287  946.293396  946.279419  946.293396  946.249634  946.278381   
    288  946.293640  946.279663  946.293640  946.249878  946.278625   

               5386        5387        5388        5389        5390  
run t                                                                
0   0    940.156982  942.093750  940.156982  942.093750  940.156982  
    1    940.156982  942.093750  940.156982  942.093750  940.156982  
    2    940.156982  942.093750  940.156982  942.093750  940.156982  
    3    940.156982  942.093750  940.156982  942.093750  940.156982  
    4    940.156982  942.093750  940.156982  942.093750  940.156982  
...             ...         ...         ...         ...         ...  
2   284  946.248962  946.277710  946.248962  946.277710  946.248962  
    285  946.249207  946.277954  946.249207  946.277954  946.249207  
    286  946.249390  946.278198  946.249390  946.278198  946.249390  
    287  946.249634  946.278381  946.249634  946.278381  946.249634  
    288  946.249878  946.278625  946.249878  946.278625  946.249878  

[867 rows x 5391 columns]

In [7]:
gpr.data_y

0           1           2           3           4     \
run t                                                                 
0   0    940.156250  939.428467  939.091248  939.405579  940.144287   
    1    940.156250  939.428467  939.091248  939.405579  940.144287   
    2    940.156250  939.428467  939.091248  939.405579  940.144287   
    3    940.156250  939.428467  939.091248  939.405579  940.144287   
    4    940.156250  939.428467  939.091248  939.405579  940.144287   
...             ...         ...         ...         ...         ...   
2   284  945.061401  945.061401  945.061401  945.061401  945.061401   
    285  945.061462  945.061462  945.061523  945.061523  945.061523   
    286  945.061523  945.061584  945.061584  945.061584  945.061584   
    287  945.061646  945.061646  945.061646  945.061646  945.061646   
    288  945.061707  945.061707  945.061707  945.061707  945.061707   

               5           6           7           8           9     ...  \
run t                                                                ...   
0   0    938.707214  938.954712  938.707214  937.979431  936.749390  ...   
    1    938.707214  938.954712  938.707214  937.979431  936.749390  ...   
    2    938.707214  938.954712  938.707214  937.979431  936.749390  ...   
    3    938.707214  938.954712  938.707214  937.979431  936.749390  ...   
    4    938.707214  938.954712  938.707214  937.979431  936.749390  ...   
...             ...         ...         ...         ...         ...  ...   
2   284  945.061401  945.061401  945.061401  945.061401  945.061401  ...   
    285  945.061462  945.061462  945.061462  945.061462  945.061462  ...   
    286  945.061523  945.061523  945.061523  945.061523  945.061523  ...   
    287  945.061584  945.061584  945.061584  945.061584  945.061584  ...   
    288  945.061646  945.061646  945.061646  945.061646  945.061646  ...   

               5381        5382        5383        5384        5385  \
run t                                                                 
0   0    943.062500  942.710205  942.750000  943.062500  943.062500   
    1    943.062500  942.710205  942.750000  943.062500  943.062500   
    2    943.062500  942.710205  942.750000  943.062500  943.062500   
    3    943.062500  942.710205  942.750000  943.062500  943.062500   
    4    943.062500  942.710205  942.750000  943.062500  943.062500   
...             ...         ...         ...         ...         ...   
2   284  946.253662  946.249023  946.250977  946.245178  946.247314   
    285  946.253906  946.249268  946.251160  946.245422  946.247498   
    286  946.254150  946.249512  946.251404  946.245667  946.247742   
    287  946.254395  946.249756  946.251648  946.245850  946.247986   
    288  946.254578  946.249939  946.251892  946.246094  946.248169   

               5386        5387        5388        5389        5390  
run t                                                                
0   0    943.062500  943.718750  943.406250  945.031250  944.161865  
    1    943.062500  943.718750  943.406250  945.031250  944.161865  
    2    943.062500  943.718750  943.406250  945.031250  944.161865  
    3    943.062500  943.718750  943.406250  945.031250  944.161865  
    4    943.062500  943.718750  943.406250  945.031250  944.161865  
...             ...         ...         ...         ...         ...  
2   284  946.240173  946.243164  946.232788  946.221924  946.210327  
    285  946.240417  946.243408  946.232971  946.222107  946.210510  
    286  946.240601  946.243652  946.233215  946.222351  946.210754  
    287  946.240845  946.243835  946.233459  946.222534  946.210938  
    288  946.241089  946.244080  946.233643  946.222778  946.211182  

[867 rows x 5391 columns]